In [1]:
from dolfinx import *
import numpy as np
from contextlib import ExitStack
import dolfinx
import numpy as np

from dolfinx import la
from dolfinx.fem import (DirichletBC, Function, VectorFunctionSpace,
                         apply_lifting, assemble_matrix, assemble_vector,
                         locate_dofs_geometrical, set_bc)
from dolfinx.io import XDMFFile
from dolfinx.mesh import CellType, GhostMode, create_box
from ufl import (Identity, SpatialCoordinate, TestFunction, TrialFunction,
                 as_vector, dx, grad, inner, sym, tr)
import ufl
from mpi4py import MPI
from petsc4py import PETSc
from slepc4py import SLEPc


In [2]:
mesh = create_box(
    MPI.COMM_WORLD, [np.array([0.0, 0.0, 0.0]),
                     np.array([1.0, 1.0, 0.01])], [20, 20, 2],
    CellType.tetrahedron, GhostMode.shared_facet)

In [3]:

# Elasticity parameters
E = 70e9
nu = 0.23
rho = 2500
mu = E / (2.0 * (1.0 + nu))
lmbda = E * nu / ((1.0 + nu) * (1.0 - 2.0 * nu))

In [4]:
# Create function space
V = VectorFunctionSpace(mesh, ("Lagrange", 1))

# Define variational problem
u_tr = TrialFunction(V)
u_test = TestFunction(V)


In [5]:
#strain function
def epsilon(u):
    return 0.5*(ufl.nabla_grad(u)+ufl.nabla_grad(u).T)
# stress function
def sigma(u):
    return lmbda * ufl.div(u)*ufl.Identity(3)+2*mu*epsilon(u)

In [6]:
# forms and matricis
a_form = ufl.inner(sigma(u_tr),epsilon(u_test))*ufl.dx
m_form = rho*ufl.inner(u_tr,u_test)*ufl.dx

In [13]:
K_mat = dolfinx.fem.assemble_matrix(a_form)
K_mat.assemble()


In [14]:
M_mat = dolfinx.fem.assemble_matrix(m_form)
M_mat.assemble()

In [15]:
K_mat.getSize()

(25215, 25215)

In [16]:
ki, kj, kv = K_mat.getValuesCSR()
Mi,Mj,Mv = M_mat.getValuesCSR() 

In [17]:
# kmat_numpy = kmat_numpy.getDenseArray()
import scipy.linalg as splg
from scipy.sparse import linalg
import scipy.sparse as spsparse

In [18]:
Ksparse = spsparse.csr_matrix((kv,kj,ki))
Msparse = spsparse.csr_matrix((Mv,Mj,Mi))

In [19]:
# check if symmetric
print((np.abs(Ksparse - Ksparse.T)>1e-10).nnz == 0)
print((np.abs(Msparse - Msparse.T)>1e-10).nnz == 0)

False
True


In [22]:
# set up eigensolver parameters
eigensolver = SLEPc.EPS().create(MPI.COMM_WORLD)
eigensolver.setOperators(K_mat,M_mat)
eigensolver.setDimensions(nev=10) # returns 10 values
# eigensolver.setProblemType(2) # Generalized Hermitian Eigensolver Problem
eigensolver.setWhichEigenpairs(2) # smallest eigenvalues
eigensolver.setFromOptions() # applies options
eigensolver.view() # checks results


EPS Object: 1 MPI processes
  type: krylovschur
    0% of basis vectors kept after restart
    using the locking variant
  problem type: not yet set
  selected portion of the spectrum: smallest eigenvalues in magnitude
  postprocessing eigenvectors with purification
  number of eigenvalues (nev): 10
  number of column vectors (ncv): -2
  maximum dimension of projected problem (mpd): -2
  maximum number of iterations: -2
  tolerance: -2.
  convergence test: relative to the eigenvalue
BV Object: 1 MPI processes
  type: svec
  0 columns of global length -1
  vector orthogonalization method: classical Gram-Schmidt
  orthogonalization refinement: if needed (eta: 0.7071)
  block orthogonalization method: GS
  doing matmult as a single matrix-matrix product
DS Object: 1 MPI processes
  type: nhep
ST Object: 1 MPI processes
  type: shift
  shift: 0.
  number of matrices: 2
  all matrices have unknown nonzero pattern
  KSP Object: (st_) 1 MPI processes
    type: preonly
    maximum iterations=1

In [ ]:
%time eigensolver.solve()